## 0. 导入依赖

In [ ]:
import os
import warnings
from pickle import load
from argparse import Namespace
from configparser import ConfigParser
from importlib.resources import files

import pandas as pd
from torch.utils.data import  Subset

from transformers import (
    GPT2LMHeadModel,
    GPT2Config,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from transformers.trainer_callback import EarlyStoppingCallback

from MiCoGPT.utils.pretrain import attach_gated_prior_to_gpt2, summarize_gate, PriorDiagnosticsCallback
from MiCoGPT.utils.tools import split_train_val_by_project

warnings.filterwarnings("ignore")


## 1. 基本参数设置

In [5]:
args = Namespace(
    input="../data/try2_withCC/ResMicroDB_90338.pkl",
    output="../models/pretrain_ResMicroDB_90338_GATED",
    log="../logs/pretrain_ResMicroDB_90338_GATED",
)

G_MIN = 0.10      # 门控先验的最小权重（至少注入 10% 的先验）
INIT_W = 0.50     # 初始先验权重（介于 g_min 和 1 之间）
VAL_RATIO = 0.10  # 验证集比例

## 2. 载入语料库

In [6]:
all_corpus = load(open(args.input, "rb"))

# 选择 Split_Group 为 A 的样本进行训练
corpus = all_corpus.subset_by_metadata(lambda df: df["Split_Group"] == "A")
tokenizer = all_corpus.tokenizer

print("Number of samples in all_corpus:", len(all_corpus))
print("Number of samples in corpus:", len(corpus))
print(all_corpus.metadata["Split_Group"].value_counts())
print("Tokenizer vocab size:", tokenizer.vocab_size)

Number of samples in all_corpus: 90338
Number of samples in corpus: 74557
Split_Group
A    74557
B    13901
C     1880
Name: count, dtype: int64
Tokenizer vocab size: 1121


## 3. 构建 GPT2Config

In [7]:
cfg = ConfigParser()
cfg.read(files("MiCoGPT")/"resources/config.ini")

gpt2_config_dict = {
    "model_type":   cfg.get("GPT2", "model_type"),
    "vocab_size":   tokenizer.vocab_size,
    "n_positions":  cfg.getint("GPT2", "n_positions"),
    "n_embd":       cfg.getint("GPT2", "n_embd"),
    "n_layer":      cfg.getint("GPT2", "n_layer"),
    "n_head":       cfg.getint("GPT2", "n_head"),
    "bos_token_id": tokenizer.bos_token_id,
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.pad_token_id,
}
config = GPT2Config(**gpt2_config_dict)
config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 2,
  "embd_pdrop": 0.1,
  "eos_token_id": 3,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 256,
  "n_head": 8,
  "n_inner": null,
  "n_layer": 8,
  "n_positions": 512,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.33.3",
  "use_cache": true,
  "vocab_size": 1121
}

## 4. 构建 TrainingArguments

In [ ]:
training_args_dict = {
    "do_train": True,
    "do_eval": True,
    "evaluation_strategy": "steps",
    "eval_steps": cfg.getint("pretrain", "eval_steps"),
    "save_strategy": "steps",
    "save_steps": cfg.getint("pretrain", "save_steps"),
    "group_by_length": False,
    "length_column_name": "length",
    "disable_tqdm": False,
    "learning_rate": cfg.getfloat("pretrain", "learning_rate"),
    "lr_scheduler_type": "linear",
    "warmup_steps": cfg.getint("pretrain", "warmup_steps"),
    "weight_decay": cfg.getfloat("pretrain", "weight_decay"),
    "per_device_train_batch_size": cfg.getint("pretrain", "per_device_train_batch_size"),
    "num_train_epochs": cfg.getint("pretrain", "num_train_epochs"),
    "logging_steps": cfg.getint("pretrain", "logging_steps"),
    "output_dir": f"{args.log}/pretrain_checkpoints",
    "logging_dir": args.log,
    "load_best_model_at_end": True,
}
training_args = TrainingArguments(**training_args_dict)
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=500,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=True,
half_precision_backend=auto,
hub_always_push=False,
hub_model

## 5. 构建 collator

In [9]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

## 6. 构建模型

In [10]:
model = GPT2LMHeadModel(config)
model.train()
print("Training from scratch.")

Training from scratch.


## 7. 挂载 gated prior

In [14]:
npz_path = files("MiCoGPT")/"resources"/"genus_embeddings_256.npz"
genus_token_ids, missing = attach_gated_prior_to_gpt2(
    model=model,
    tokenizer=tokenizer,
    npz_path=npz_path,
    g_min=G_MIN,
    init_w=INIT_W,
)
print(f"[gated prior] genus_token_ids={len(genus_token_ids)}, missing={len(missing)}")
summarize_gate(model, topk=10)

[prior] npz genus 总数: 1117
[prior] 写入 prior 的 unique token_id 数: 1117
[prior] missing/unk genus 数: 0
[gated prior] genus_token_ids=1117, missing=0
[gate] g_min=0.1, prior_nonzero_tokens=1117
[gate] w mean=0.5000, std=0.0000, min=0.5000, max=0.5000
[gate] top using prior token_ids: [9, 13, 6, 10, 12, 8, 4, 5, 7, 11]
[gate] top weights: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


## 8. 划分 train/val

In [ ]:
train_set, val_set = split_train_val_by_project(
    corpus,
    val_ratio=VAL_RATIO,
    project_col="Project_ID",
    random_state=42,
)

## 9. callbacks

In [ ]:
# callbacks = [
#     EarlyStoppingCallback(early_stopping_patience=3),
#     GateMonitorCallback(every_eval=True, topk=10),   # 可注释掉
# ]

callbacks = [
    EarlyStoppingCallback(early_stopping_patience=10),
    # 检查 gate 统计是否正常，并且做一个消融实验
    PriorDiagnosticsCallback(tokenizer=tokenizer, topk=10, do_ablation=True),
]


In [ ]:
# ========= Trainer =========
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    data_collator=data_collator,
    callbacks=callbacks,
)

print("Start training...")
trainer.train()

# ========= 保存模型 =========
os.makedirs(args.output, exist_ok=True)
trainer.save_model(args.output)
print("Model saved to:", args.output)

# ========= 保存训练日志 =========
logs = pd.DataFrame(trainer.state.log_history)
os.makedirs(args.log, exist_ok=True)
log_path = os.path.join(args.log, "pretrain_log.csv")
logs.to_csv(log_path, index=False)
print("Logs saved to:", log_path)
